# Geo Skills Building: Planet Data
Date: May 25, 2021 <br/>
Led by Erica Carcelen and Scott Cunningham <br/>
Code pulled from [ordering_delivery.ipynb](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/orders/ordering_and_delivery.ipynb) in the [Planet Labs repository]( https://github.com/planetlabs/notebooks/tree/master/jupyter-notebooks)


## Background
#### NASA Data Buy
In April 2020, NASA expanded their subscription with Planet to provide imagery to all NASA-funded research programs (including DEVELOP!) This subscription includes access to the PlanetScope & RapidEye imagery for dates more than 30 days ago. More information about these products can be found [here](https://assets.planet.com/docs/Planet_Combined_Imagery_Product_Specs_letter_screen.pdf).<br/>
Because participants are with the program for such a short period of time, access to NASA's subscription will be facilitated through the Fellow. If your team needs access and you have not setup an account through NASA's subscription, the Fellow should contact Geoinformatics or NPO. The Fellow will place imagery orders for their team and manage access to the imagery. Project partners cannot receive the original, raw imagery but they can receive products (map products, data products - NDVI, NWI, etc.). Please review and share the terms of use with your team. This notebook outlines the process for the Fellow to order imagery for their team through Google Earth Engine. 
#### Norwegian Data Buy - NICFI Program
Under Norway's International Climate & Forests Initiative (NICFI), anyone can access Planet's high-resolution, analysis-ready mosaics of the world's tropics. More information about the NICFI program can be found [here](https://assets.planet.com/docs/NICFI_UserGuidesFAQ.pdf). Because anyone can access this imagery, you or your participants can setup an account by filling out [this form](https://www.planet.com/nicfi/) and downloading imagery with the [QGIS or ArcGIS plugin](https://developers.planet.com/integrations/). Please note that if you've already set an up account with NASA's subscription, you must sign-up for an account under NICFI with a <i>different email</i>. Planet plans to enable NICFI download through Google Earth Engine, but it is currently not available.

## Setup
Fellows will facilitate ordering Planet imagery under NASA's subscription for their team. You will work with their team to get the ids for the imagery needed and the Fellow will submit an order to Planet's API under their NASA account. With the code in this notebook, you'll submit an order and Planet will deliver your order as a GEE asset to a Google Cloud Project. As members of the Google Cloud Project, your team can then access the imagery for the duration of the term. Below are the steps to get setup, which were pulled from [Planet's GEE Setup Guide](https://developers.planet.com/docs/integrations/gee/quickstart/).
#### Enable Google Cloud Projects
To use Planet's GEE delivery, your GEE account must be integrated with Google Cloud Projects.
Upgrade your existing GEE account by submitting a request to Google with [this form](https://docs.google.com/forms/d/e/1FAIpQLSdUR5j6lfcsTh3k5H1RJYKQKLRgSheAzKLkYfDdnx9YQ6wUtw/viewform?usp=sf_link).
#### Create Google Cloud Project
Google does not notify you when your account has been upgraded, but it should only take a few days. <br/>
Once your account has been upgraded, a pop-up will appear the first time you open [GEE Code Editor](https://code.earthengine.google.com/) prompting you to select or create a cloud project.<br/> 
You can also create a project by following [this link](https://console.cloud.google.com/projectcreate). <br/>
More information about managing cloud projects can be found [here](https://cloud.google.com/resource-manager/docs/creating-managing-projects).
#### Enable Earth Engine API
Now you need to enable the Earth Engine API for your cloud project. <br/> 
Navigate to the [Earth Engine API on the Google Cloud Console](https://console.cloud.google.com/apis/library/earthengine.googleapis.com). <br/>Select your cloud project from the drop down at the top, and click `Enable`. <br/>
Now your Code Editor will look like normal, but the top right icon will have your cloud project name instead of your email and there will be a `Cloud Assets` section on your `Assets` tab. 
#### Grant Planet's Service Account Access
The final step is to grant Planet access to your cloud project so that they can deliver the imagery as a Cloud Asset. <br/>
Navigate to your [Google Cloud Platform Console](https://console.cloud.google.com/home/dashboard).<br/>
Select the `IAM & Admin` page.<br/>
Check that the correct cloud project is in the drop-down at the top of the page.<br/>
On the `IAM` page, select `Add` and paste <b>`planet-gee-uploader@planet-earthengine-staging.iam.gserviceaccount.com`</b> as the New Member and select <b>`Earth Engine Resource Writer`</b> as the `Role` <br/>
You can also add your participants to the cloud project as `Editors` on this page. More information about the various roles can be found [here](https://cloud.google.com/iam/help/roles/predefined-roles-reference?authuser=1&_ga=2.30224058.-424489327.1609826186)

## Select Imagery
#### Prepare Delivery to GEE
To have Planet imagery delivered to GEE, there must be an exisiting image collection in your Cloud Assets where Planet can put the images. 
In [GEE Code Editor](https://code.earthengine.google.com/), go the `Assets` tab. <br/>
Click `New`, select `Image collection` from the drop-down menu, and give it a descriptive name (no spaces).
#### Get the Order Information
The remaining information needed for your order are the API Key for your account and the Image IDs for the imagery you're ordering. <br/>
Retrieve your API Key (copy to your clipboard) from the Profile page on your [Planet Account](https://www.planet.com/account/#/profile)<br/>
You or your participants can browse imagery using [Planet Explorer](https://www.planet.com/explorer/) or the ArcGIS/QGIS plug-in. Retrieve the scene ids and store them in a list for later.

## Order Imagery to GEE
Now that you're setup and have the information for your order, use the code below to input an order. This code engages with Planet's Order API and submits the order as a JSON.<br/>
My example will submit an order with the following information:<br/>
`2 Planet Images:"20210316_185700_36_241c", "20210317_180625_00_2431"`<br/>
`GEE Project: ee-coastalcawater`<br/>
`GEE Image Collection: Carmel_March162021`

In [ ]:
#import packages required for the code
import json
import os
import pathlib
import time

import requests
from requests.auth import HTTPBasicAuth

#### Authenticating
Setting up the API Key

In [ ]:
# Define API Key as environment variable
## Enter your API Key on the following line and uncomment it:
#os.environ['PL_API_KEY'] = 'YOUR API KEY HERE'

In [ ]:
# Setup the API Key from the 'PL_API_Key' environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

In [ ]:
# Define the Planet Data orders URL
orders_url = 'https://api.planet.com/basemaps/v1/mosaics'

In [ ]:
#Curl example
#To check your orders list and make sure you have the permissions you need, uncomment the following line to run curl
!curl -L -H "Authorization: api-key $PLANET_API_KEY" $orders_url

We are using requests to communicate with the orders v2 API. To verify that we've setup the API key correctly and we're connected to our Planet account, we'll check our orders list.

If you have setup correctly, the output of the below code chunk should be: <Response [200]> <br/>
To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/docs/orders/reference/#operation/listOrders) AOI reference.

In [ ]:
# Verify connection to Planet account
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

Now we can access orders that have been previously made with this account. Below we'll see how many orders have been made. If you've never ordered imagery before, the output value should be 0! You can verify this value by looking at your account on the Planet website

In [ ]:
# Access list of orders associated with the account
orders = response.json()['orders']
len(orders)

#### Place Order

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

In [ ]:
# Define request parameters
## Here is where you'll modify information for the images you are ordering
request = {
    #order specifications
    "name": "Carmel March 16 Breach Event", #name for your order
    "products": [
        {
            "item_ids": [
                "20210316_185700_36_241c", #ids for images you are ordering
                "20210317_180625_00_2431"
            ],
            "item_type": "PSScene4Band", #image type
            "product_bundle": "analytic_sr_udm2" #product bundle
        }
    ],
    #delivery specifications
    "delivery": {
        "google_earth_engine": {
            "project": "ee-coastalcawater", #cloud project name
            "collection": "Carmel_March162021" #empty collection name
        }
    }
}

In [ ]:
# Define function to place order
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

In [ ]:
# Send request and place order
## if successul, the output will be <Response [202]> and your Order Id
order_url = place_order(request, auth)

#### Poll for Order Success
We have the id for our order, but we don't know if it's been successfully fulfilled. The following code will check the status of the order every 10 seconds. If it times out before returning "success", "failed", or "partial", you can re-run the code to check the status or increase the time step. 

In [ ]:
# Define function to check order status
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)

poll_for_success(order_url, auth)

#### Check Results
Now we can check the results of our order request! This code access the results of your order and lists the filenames of the results. <br/>
If you had a "partial" return message, you can check which images were successfully delivered.<br/>
You will get an error if you had a "failed" return message because no results were produced.

In [ ]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']

[r['name'] for r in results]

## Your team is ready to go!
Once successful, your Planet imagery is delivered straight to the assets of your GEE Cloud Project! As members of your cloud project, your participants should automatically be able to access the image collection and call it in to a GEE script.